# 뉴럴 네트워크 학습 알고리즘 구현

## Import modules

In [2]:
import tensorflow as tf

## 하이퍼파라미터 정의

In [3]:
EPOCHS = 10

## 네트워크 구조 정의

In [7]:
class Mymodel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
        self.dense1 = tf.keras.layers.Dense(32, activation='relu')
        self.dense2 = tf.keras.layers.Dense(64, activation='relu')
        self.dense3 = tf.keras.layers.Dense(128, activation='relu')
        self.dense4 = tf.keras.layers.Dense(256, activation='relu')
        self.dense5 = tf.keras.layers.Dense(10, activation='softmax')
    
    def call(self, x, training=None, mask=None):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        return self.dense5(x)
        

## 학습 함수 구현

In [9]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images) # BatchSize x 10(Classes)
        loss = loss_object(labels, predictions)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

## 테스트 함수 구현

In [25]:
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images) # BatchSize x 10(Classes)
    loss = loss_object(labels, predictions)
    
    test_loss(loss)
    test_accuracy(labels, predictions)

## 데이터 불러오기

In [16]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1024).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 모델 생성

In [17]:
model = Mymodel()

## 손실함수 및 최적화 알고리즘 정의

In [20]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## 성능 지표 정의

In [21]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 구현

In [26]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
    
    for images, labels in test_ds:
        test_step(model, images, labels, loss_object, test_loss, test_accuracy)
        
    template = 'Epoch: {}, Loss": {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch +1,
                         train_loss.result(),
                         train_accuracy.result() * 100,
                         test_loss.result(),
                         test_accuracy.result() * 100))
    
    train_loss.reset_states()
    train_accuracy.reset_states()

    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch: 1, Loss": 0.214940145611763, Accuracy: 93.43416595458984, Test Loss: 0.17570316791534424, Test Accuracy: 94.80000305175781
Epoch: 2, Loss": 0.11156249046325684, Accuracy: 96.5250015258789, Test Loss: 0.13754898309707642, Test Accuracy: 95.95000457763672
Epoch: 3, Loss": 0.09293366223573685, Accuracy: 97.11666870117188, Test Loss: 0.1449742168188095, Test Accuracy: 95.77999877929688
Epoch: 4, Loss": 0.08084052801132202, Accuracy: 97.47000122070312, Test Loss: 0.11921720206737518, Test Accuracy: 96.6500015258789
Epoch: 5, Loss": 0.07105067372322083, Accuracy: 97.76667022705078, Test Loss: 0.10743945837020874, Test Accuracy: 97.0
Epoch: 6, Loss": 0.062200963497161865, Accuracy: 98.06832885742188, Test Loss: 0.14176714420318604, Test Accuracy: 96.1500015258789
Epoch: 7, Loss": 0.05457993969321251, Accuracy: 98.28166961669922, Test Loss: 0.11876516044139862, Test Accuracy: 96.66999816894531
Epoch: 8, Loss": 0.05075087025761604, Accuracy: 98.36500549316406, Test Loss: 0.11525387316942